In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [4]:
#!/usr/bin/env python3
# Kalshi API Interface for Weather Markets

import requests
import json
import time
import logging
from typing import List, Dict, Any, Optional, Union

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger("kalshi_client")

class KalshiClient:
    """Client for interacting with the Kalshi API"""
    
    BASE_URL = "https://api.elections.kalshi.com/trade-api/v2"
    
    def __init__(self, email: str, password: str):
        """Initialize the Kalshi client with credentials"""
        self.email = email
        self.password = password
        self.token = None
        self.headers = {
            "Content-Type": "application/json",
            "Accept": "application/json"
        }
    
    def authenticate(self) -> str:
        """Authenticate with Kalshi API and get access token"""
        url = f"{self.BASE_URL}/login"
        payload = {
            "email": self.email,
            "password": self.password
        }
        
        try:
            logger.info(f"Authenticating with Kalshi API at {url}")
            response = requests.post(url, json=payload, headers=self.headers)
            
            # Print response status for debugging
            logger.info(f"Authentication response status: {response.status_code}")
            
            # Try to get response body even if there's an error
            try:
                response_data = response.json()
                logger.debug(f"Authentication response data: {json.dumps(response_data)}")
            except:
                logger.debug(f"Could not parse response as JSON, raw content: {response.text[:200]}...")
            
            response.raise_for_status()
            
            data = response.json()
            self.token = data.get("token")
            
            if not self.token:
                raise ValueError("Authentication successful but no token received")
            
            # Update headers with token
            self.headers["Authorization"] = f"Bearer {self.token}"
            logger.info("Authentication successful")
            return self.token
            
        except requests.exceptions.RequestException as e:
            logger.error(f"Authentication failed: {str(e)}")
            raise
    
    def get_markets(self, max_retries: int = 3, initial_delay: float = 1.0, 
                 params: Optional[Dict[str, Any]] = None) -> List[Dict[str, Any]]:
        """Get all markets with retry logic"""
        if not self.token:
            self.authenticate()
        
        url = f"{self.BASE_URL}/markets"
        retries = 0
        delay = initial_delay
        
        # Log request details
        logger.info(f"Fetching markets from {url}")
        if params:
            logger.info(f"With parameters: {params}")
        
        while retries <= max_retries:
            try:
                response = requests.get(url, headers=self.headers, params=params, timeout=15)
                
                # Print response status for debugging
                logger.info(f"Markets API response status: {response.status_code}")
                
                # Try to log response content even if status code is not success
                try:
                    response_data = response.json()
                    logger.debug(f"API response preview: {str(response_data)[:200]}...")
                    # Log useful response metadata
                    if "markets" in response_data:
                        logger.info(f"Number of markets in response: {len(response_data['markets'])}")
                except:
                    logger.debug(f"Could not parse response as JSON, raw content: {response.text[:200]}...")
                
                # Handle service unavailability
                if response.status_code == 503:
                    logger.warning(f"Service unavailable (503), attempt {retries + 1}/{max_retries + 1}")
                    
                    if retries >= max_retries:
                        raise Exception("Kalshi API service is currently unavailable after multiple attempts")
                    
                    # Wait with exponential backoff before retrying
                    time.sleep(delay)
                    delay *= 2  # Double the delay for next retry
                    retries += 1
                    continue
                
                # Handle rate limiting
                if response.status_code == 429:
                    logger.warning(f"Rate limited (429), attempt {retries + 1}/{max_retries + 1}")
                    
                    if retries >= max_retries:
                        raise Exception("Kalshi API request rate limit exceeded after multiple attempts")
                    
                    # Get retry-after header if available or use exponential backoff
                    retry_after = response.headers.get('Retry-After')
                    wait_time = int(retry_after) if retry_after else delay
                    
                    time.sleep(wait_time)
                    delay *= 2
                    retries += 1
                    continue
                
                # Handle other error status codes
                response.raise_for_status()
                
                data = response.json()
                markets = data.get("markets", [])
                logger.info(f"Successfully retrieved {len(markets)} markets")
                return markets
                
            except requests.exceptions.RequestException as e:
                logger.error(f"Error retrieving markets: {str(e)}")
                
                # Handle connection issues
                if isinstance(e, (requests.exceptions.ConnectionError, requests.exceptions.Timeout)):
                    logger.warning(f"Connection issue, attempt {retries + 1}/{max_retries + 1}")
                    
                    if retries >= max_retries:
                        raise Exception("Cannot connect to Kalshi API after multiple attempts")
                    
                    time.sleep(delay)
                    delay *= 2
                    retries += 1
                    continue
                
                raise
        
        raise Exception("Maximum retries exceeded for Kalshi API request")
    
    def get_weather_markets_through_events(self) -> List[Dict[str, Any]]:
        """Alternative approach: Get weather markets through events endpoint"""
        if not self.token:
            self.authenticate()
        
        try:
            # Step 1: Get all events
            events_url = f"{self.BASE_URL}/events"
            events_response = requests.get(events_url, headers=self.headers)
            events_response.raise_for_status()
            
            events_data = events_response.json()
            
            # Step 2: Filter for weather-related events
            weather_events = []
            for event in events_data.get("events", []):
                title_lower = (event.get("title", "") or "").lower()
                category_lower = (event.get("category", "") or "").lower()
                
                if (category_lower == "weather" or
                    "weather" in title_lower or
                    "rain" in title_lower or
                    "temperature" in title_lower or
                    "precipitation" in title_lower):
                    weather_events.append(event)
            
            # Step 3: Get markets for each weather event
            weather_markets = []
            
            for event in weather_events:
                event_markets_url = f"{self.BASE_URL}/events/{event.get('ticker')}/markets"
                event_markets_response = requests.get(event_markets_url, headers=self.headers)
                
                if event_markets_response.ok:
                    event_markets_data = event_markets_response.json()
                    weather_markets.extend(event_markets_data.get("markets", []))
            
            return weather_markets
            
        except requests.exceptions.RequestException as e:
            logger.error(f"Error fetching weather markets through events: {str(e)}")
            raise
    
    def filter_weather_markets(self, markets: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        """Filter markets to only include weather-related ones"""
        weather_markets = []
        
        for market in markets:
            # Check for weather category
            if market.get("category") == "WEATHER":
                weather_markets.append(market)
                continue
            
            # Check for weather tag
            tags = market.get("tags", [])
            if isinstance(tags, list) and "weather" in [tag.lower() for tag in tags if tag]:
                weather_markets.append(market)
                continue
            
            # Check if the title or subtitle contains weather-related keywords
            title = (market.get("title") or "").lower()
            subtitle = (market.get("subtitle") or "").lower()
            
            weather_keywords = ["rain", "snow", "temperature", "precipitation", 
                               "hurricane", "storm", "climate"]
            
            if any(keyword in title or keyword in subtitle for keyword in weather_keywords):
                weather_markets.append(market)
        
        return weather_markets
    
    def get_weather_markets(self, use_events_endpoint: bool = True) -> List[Dict[str, Any]]:
        """Get all weather-related markets from Kalshi"""
        try:
            # Try the main approach first
            try:
                # Fetch all markets
                markets = self.get_markets()
                
                # Filter for weather markets
                weather_markets = self.filter_weather_markets(markets)
                
                return weather_markets
                
            except Exception as e:
                error_msg = str(e)
                
                # If the markets endpoint fails and use_events_endpoint is True, try the alternative approach
                if use_events_endpoint and ("service is currently unavailable" in error_msg or 
                                           "Cannot connect to Kalshi API" in error_msg):
                    logger.info("Markets endpoint failed, trying alternative approach through events...")
                    return self.get_weather_markets_through_events()
                
                raise
                
        except Exception as e:
            logger.error(f"Error fetching weather markets: {str(e)}")
            raise

def try_all_api_endpoints(email, password):
    """Try multiple possible API endpoints to find the working one"""
    
    # List of potential API base URLs to try
    potential_endpoints = [
        "https://api.elections.kalshi.com/trade-api/v2",
        "https://api.kalshi.com/trade-api/v2",
        "https://api.kalshi.com/v1",
        "https://trading-api.kalshi.com/v1"
    ]
    
    # Try each endpoint
    for endpoint in potential_endpoints:
        try:
            print(f"\nTrying API endpoint: {endpoint}")
            # Temporarily modify the base URL
            KalshiClient.BASE_URL = endpoint
            
            # Create client and attempt authentication
            client = KalshiClient(email, password)
            client.authenticate()
            
            print(f"✅ Successfully connected to: {endpoint}")
            return client
            
        except Exception as e:
            print(f"❌ Failed with endpoint {endpoint}: {str(e)}")
            continue
    
    raise Exception("All Kalshi API endpoints failed. Please check your credentials or try again later.")

def main():
    """Main function to run the script"""
    try:
        # Replace with your actual Kalshi credentials
        email = "happynowbtodd@gmail.com"
        password = "F1shF1sh**"
        
        print("Attempting to connect to Kalshi API (trying multiple endpoints)...")
        
        # Try to find a working endpoint
        client = try_all_api_endpoints(email, password)
        
        # If we get here, we found a working endpoint
        print(f"\nFetching weather markets from {client.BASE_URL}...")
        
        # Try to get weather markets with query parameter first
        try:
            # Try different approaches
            weather_api_approaches = [
                {"name": "Method 1: markets with category param", 
                 "call": lambda: client.get_markets(params={"category": "WEATHER"})},
                
                {"name": "Method 2: events with category param", 
                 "call": lambda: client.get_weather_markets_through_events(category_param=True)},
                
                {"name": "Method 3: all markets, local filtering", 
                 "call": lambda: client.get_weather_markets()}
            ]
            
            # Try each approach
            weather_markets = None
            for approach in weather_api_approaches:
                try:
                    print(f"\nTrying {approach['name']}...")
                    weather_markets = approach["call"]()
                    print(f"✅ Success with {approach['name']}")
                    break
                except Exception as e:
                    print(f"❌ Failed: {str(e)}")
            
            if weather_markets is None:
                raise Exception("All methods to fetch weather markets failed")
        
        except Exception as e:
            print(f"Error fetching weather markets: {str(e)}")
            return None
        
        # Display results
        print(f"\nFound {len(weather_markets)} weather markets:")
        for index, market in enumerate(weather_markets, 1):
            print(f"{index}. {market.get('title')} ({market.get('ticker')})")
        
        # Optionally write to file
        # with open('weather-markets.json', 'w') as f:
        #     json.dump(weather_markets, f, indent=2)
        
        # Success message
        print("\nSuccessfully retrieved weather markets from Kalshi API!")
        print(f"Working API endpoint: {client.BASE_URL}")
        
        return weather_markets
        
    except Exception as e:
        print(f"\nError in main function: {str(e)}")
        
        print("\nTROUBLESHOOTING TIPS:")
        print("1. Double-check your Kalshi credentials")
        print("2. Verify your internet connection")
        print("3. Check if Kalshi's service status page reports any outages")
        print("4. The API structure might have changed - check Kalshi's latest documentation")
        print("5. Contact Kalshi support for assistance")
        
        return None

if __name__ == "__main__":
    main()

2025-04-08 21:18:29,909 - kalshi_client - INFO - Authenticating with Kalshi API at https://api.elections.kalshi.com/trade-api/v2/login


Attempting to connect to Kalshi API (trying multiple endpoints)...

Trying API endpoint: https://api.elections.kalshi.com/trade-api/v2


2025-04-08 21:18:30,199 - kalshi_client - INFO - Authentication response status: 400
2025-04-08 21:18:30,199 - kalshi_client - ERROR - Authentication failed: 400 Client Error: Bad Request for url: https://api.elections.kalshi.com/trade-api/v2/login
2025-04-08 21:18:30,200 - kalshi_client - INFO - Authenticating with Kalshi API at https://api.kalshi.com/trade-api/v2/login
2025-04-08 21:18:30,303 - kalshi_client - ERROR - Authentication failed: HTTPSConnectionPool(host='api.kalshi.com', port=443): Max retries exceeded with url: /trade-api/v2/login (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x112adb350>: Failed to resolve 'api.kalshi.com' ([Errno 8] nodename nor servname provided, or not known)"))
2025-04-08 21:18:30,304 - kalshi_client - INFO - Authenticating with Kalshi API at https://api.kalshi.com/v1/login
2025-04-08 21:18:30,313 - kalshi_client - ERROR - Authentication failed: HTTPSConnectionPool(host='api.kalshi.com', port=443): Max retries exceede

❌ Failed with endpoint https://api.elections.kalshi.com/trade-api/v2: 400 Client Error: Bad Request for url: https://api.elections.kalshi.com/trade-api/v2/login

Trying API endpoint: https://api.kalshi.com/trade-api/v2
❌ Failed with endpoint https://api.kalshi.com/trade-api/v2: HTTPSConnectionPool(host='api.kalshi.com', port=443): Max retries exceeded with url: /trade-api/v2/login (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x112adb350>: Failed to resolve 'api.kalshi.com' ([Errno 8] nodename nor servname provided, or not known)"))

Trying API endpoint: https://api.kalshi.com/v1
❌ Failed with endpoint https://api.kalshi.com/v1: HTTPSConnectionPool(host='api.kalshi.com', port=443): Max retries exceeded with url: /v1/login (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x112ae1f10>: Failed to resolve 'api.kalshi.com' ([Errno 8] nodename nor servname provided, or not known)"))

Trying API endpoint: https://trading-api.kalshi.

2025-04-08 21:18:30,433 - kalshi_client - ERROR - Authentication failed: HTTPSConnectionPool(host='trading-api.kalshi.com', port=443): Max retries exceeded with url: /v1/login (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x112ae1d50>: Failed to resolve 'trading-api.kalshi.com' ([Errno 8] nodename nor servname provided, or not known)"))


❌ Failed with endpoint https://trading-api.kalshi.com/v1: HTTPSConnectionPool(host='trading-api.kalshi.com', port=443): Max retries exceeded with url: /v1/login (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x112ae1d50>: Failed to resolve 'trading-api.kalshi.com' ([Errno 8] nodename nor servname provided, or not known)"))

Error in main function: All Kalshi API endpoints failed. Please check your credentials or try again later.

TROUBLESHOOTING TIPS:
1. Double-check your Kalshi credentials
2. Verify your internet connection
3. Check if Kalshi's service status page reports any outages
4. The API structure might have changed - check Kalshi's latest documentation
5. Contact Kalshi support for assistance


In [14]:
import requests

url = "https://api.elections.kalshi.com/trade-api/v2/events"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)
# Filter Category Climate and Weather


{"events":[{"event_ticker":"KXROBOTMARS-35","series_ticker":"KXROBOTMARS","sub_title":"Before 2035","title":"Will a humanoid robot walk on Mars before a human does?","collateral_return_type":"","mutually_exclusive":false,"category":"Science and Technology"},{"event_ticker":"KXLALEADEROUT-35","series_ticker":"KXLALEADEROUT","sub_title":"Before 2035","title":"Which of these Latin America leaders will leave office next?","collateral_return_type":"MECNET","mutually_exclusive":true,"category":"Politics"},{"event_ticker":"KXG7LEADEROUT-35","series_ticker":"KXG7LEADEROUT","sub_title":"Before 2035","title":"Which of these G7 leaders will leave office next?","collateral_return_type":"MECNET","mutually_exclusive":true,"category":"Politics"},{"event_ticker":"KXEULEADEROUT-35","series_ticker":"KXEULEADEROUT","sub_title":"Before 2035","title":"Which of these European leaders will leave office next?","collateral_return_type":"MECNET","mutually_exclusive":true,"category":"Politics"},{"event_ticker":"

In [11]:
import requests

url = "https://api.elections.kalshi.com/trade-api/v2/markets"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)


{"markets":[{"ticker":"KXCPICOREYOY-25JUN-T2.8","event_ticker":"KXCPICOREYOY-25JUN","market_type":"binary","title":"Will the rate of core CPI inflation be above 2.8% for the year ending in June 2025?","subtitle":"2.8","yes_sub_title":"Above 2.8","no_sub_title":"Above 2.8","open_time":"2025-05-14T14:00:00Z","close_time":"2025-07-15T12:25:00Z","expected_expiration_time":"2025-07-15T14:00:00Z","expiration_time":"2025-07-22T14:00:00Z","latest_expiration_time":"2025-07-22T14:00:00Z","settlement_timer_seconds":3600,"status":"initialized","response_price_units":"usd_cent","notional_value":100,"tick_size":1,"yes_bid":0,"yes_ask":0,"no_bid":100,"no_ask":100,"last_price":0,"previous_yes_bid":0,"previous_yes_ask":0,"previous_price":0,"volume":0,"volume_24h":0,"liquidity":0,"open_interest":0,"result":"","can_close_early":true,"expiration_value":"","category":"","risk_limit_cents":0,"strike_type":"greater","floor_strike":2.8,"rules_primary":"If the Consumer Price Index (CPI) for All Urban Consumers

In [13]:
# Get Events
# Get Event gets the details
# Get Markets gets the betting odds

In [15]:
import requests

# API endpoint for markets
url = "https://api.elections.kalshi.com/trade-api/v2/markets"

# Headers
headers = {"accept": "application/json"}

# Parameters - using the market ticker from your URL
params = {"ticker": "kxhighlax"}  # The market ticker from your URL

# Make the request
response = requests.get(url, headers=headers, params=params)

# Print the response
print(response.status_code)
print(response.json())

200
{'markets': [{'ticker': 'KXCPICORE-25MAY-T0.5', 'event_ticker': 'KXCPICORE-25MAY', 'market_type': 'binary', 'title': 'Will core inflation rise more than 0.5% in May?', 'subtitle': '0.5', 'yes_sub_title': 'Above 0.5%', 'no_sub_title': 'Above 0.5%', 'open_time': '2025-04-11T18:30:57Z', 'close_time': '2025-06-11T12:25:00Z', 'expected_expiration_time': '2025-06-11T14:00:00Z', 'expiration_time': '2025-06-18T14:00:00Z', 'latest_expiration_time': '2025-06-18T14:00:00Z', 'settlement_timer_seconds': 3600, 'status': 'initialized', 'response_price_units': 'usd_cent', 'notional_value': 100, 'tick_size': 1, 'yes_bid': 0, 'yes_ask': 0, 'no_bid': 100, 'no_ask': 100, 'last_price': 0, 'previous_yes_bid': 0, 'previous_yes_ask': 0, 'previous_price': 0, 'volume': 0, 'volume_24h': 0, 'liquidity': 0, 'open_interest': 0, 'result': '', 'can_close_early': True, 'expiration_value': '', 'category': '', 'risk_limit_cents': 0, 'strike_type': 'greater', 'floor_strike': 0.5, 'rules_primary': 'If the seasonally a

In [27]:
import requests

url = "https://api.elections.kalshi.com/trade-api/v2/markets"
headers = {"accept": "application/json"}

# Filter by event ticker (for all markets in a specific event)
params = {"event_ticker": "kxhighlax-25apr11"}
response = requests.get(url, headers=headers, params=params)
data = response.json()

print(f"Found {len(data['markets'])} markets for event kxhighlax")

Found 0 markets for event kxhighlax


In [30]:
import requests
url = "https://api.elections.kalshi.com/trade-api/v2/markets"
headers = {"accept": "application/json"}

# Filter by event ticker (for all markets in a specific event)
params = {"event_ticker": "kxhighlax-25apr11"}  # Changed to match the print statement
response = requests.get(url, headers=headers, params=params)
data = response.json()

print(f"Found {len(data['markets'])} markets for event KXCPICORE-11APRIL")

# Print details about each market
if 'markets' in data and data['markets']:
    for market in data['markets']:
        print(f"- {market['title']} (Ticker: {market['ticker']})")
        print(f"  Status: {market['status']}")
        print(f"  Open Time: {market['open_time']}")
        print(f"  Close Time: {market['close_time']}")
        print(f"  Rules: {market['rules_primary']}")
        print()
else:
    print("No markets found for this event or event ticker might not exist.")

Found 0 markets for event KXCPICORE-11APRIL
No markets found for this event or event ticker might not exist.


In [29]:
import requests
url = "https://api.elections.kalshi.com/trade-api/v2/markets"
headers = {"accept": "application/json"}
# Filter by market ticker
params = {"tickers": "kxhighlax-25apr11"}
response = requests.get(url, headers=headers, params=params)
data = response.json()

if 'markets' in data and data['markets']:
    print(f"Found {len(data['markets'])} markets for ticker kxhighlax")
    # Print details about the market
    market = data['markets'][0]
    print(f"Title: {market['title']}")
    print(f"Status: {market['status']}")
    print(f"Open Time: {market['open_time']}")
    print(f"Close Time: {market['close_time']}")
    print(f"Rules: {market['rules_primary']}")
    print(f"Event Ticker: {market['event_ticker']}")  # This shows the event it belongs to
else:
    print("No markets found with ticker kxhighlax")

No markets found with ticker kxhighlax


In [32]:
import requests

# Get a larger batch of markets
url = "https://api.elections.kalshi.com/trade-api/v2/markets"
headers = {"accept": "application/json"}
params = {"limit": 1000}  # Request maximum number of markets

response = requests.get(url, headers=headers, params=params)
data = response.json()

if 'markets' in data:
    # Look for markets with "Los Angeles" and "temperature" in the title
    la_temp_markets = [
        market for market in data['markets'] 
        if "los angeles" in market['title'].lower() and 
           ("temperature" in market['title'].lower() or 
            "high" in market['title'].lower() or 
            "temp" in market['title'].lower())
    ]
    
    if la_temp_markets:
        print(f"Found {len(la_temp_markets)} Los Angeles temperature-related markets:")
        for market in la_temp_markets:
            print(f"- {market['title']} (Ticker: {market['ticker']})")
            print(f"  Event Ticker: {market['event_ticker']}")
            print(f"  Status: {market['status']}")
            print(f"  Close Time: {market['close_time']}")
            print()
    else:
        print("No Los Angeles temperature markets found")
        
    # Also check for anything with "kxhighlax" in the ticker
    kx_markets = [
        market for market in data['markets'] 
        if "kxhighlax" in market['ticker'].lower() or 
           "kxhighlax" in market['event_ticker'].lower()
    ]
    
    if kx_markets:
        print(f"\nFound {len(kx_markets)} markets with kxhighlax in ticker or event_ticker:")
        for market in kx_markets:
            print(f"- {market['title']} (Ticker: {market['ticker']})")
            print(f"  Event Ticker: {market['event_ticker']}")
            print()
    else:
        print("\nNo markets found with kxhighlax in ticker or event_ticker")
else:
    print("No markets found or unexpected response format")

No Los Angeles temperature markets found

Found 6 markets with kxhighlax in ticker or event_ticker:
- Will the **high temp in LA** be >72° on Apr 12, 2025? (Ticker: KXHIGHLAX-25APR12-T72)
  Event Ticker: KXHIGHLAX-25APR12

- Will the **high temp in LA** be 71-72° on Apr 12, 2025? (Ticker: KXHIGHLAX-25APR12-B71.5)
  Event Ticker: KXHIGHLAX-25APR12

- Will the **high temp in LA** be 69-70° on Apr 12, 2025? (Ticker: KXHIGHLAX-25APR12-B69.5)
  Event Ticker: KXHIGHLAX-25APR12

- Will the **high temp in LA** be 67-68° on Apr 12, 2025? (Ticker: KXHIGHLAX-25APR12-B67.5)
  Event Ticker: KXHIGHLAX-25APR12

- Will the **high temp in LA** be 65-66° on Apr 12, 2025? (Ticker: KXHIGHLAX-25APR12-B65.5)
  Event Ticker: KXHIGHLAX-25APR12

- Will the **high temp in LA** be <65° on Apr 12, 2025? (Ticker: KXHIGHLAX-25APR12-T65)
  Event Ticker: KXHIGHLAX-25APR12



In [33]:
import requests

# Get details for the event
event_ticker = "KXHIGHLAX-25APR12"
url = f"https://api.elections.kalshi.com/trade-api/v2/events/{event_ticker}"
headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)
event_data = response.json()

print(f"Event details for {event_ticker}:")
print(f"Title: {event_data.get('title')}")
print(f"Category: {event_data.get('category')}")
print()

# Get details for each market in this event
markets_url = "https://api.elections.kalshi.com/trade-api/v2/markets"
markets_params = {"event_ticker": event_ticker}
markets_response = requests.get(markets_url, headers=headers, params=markets_params)
markets_data = markets_response.json()

if 'markets' in markets_data:
    print(f"All markets for LA temperature on April 12, 2025:")
    for market in markets_data['markets']:
        print(f"- {market['title']}")
        print(f"  Ticker: {market['ticker']}")
        print(f"  Status: {market['status']}")
        print(f"  Close Time: {market['close_time']}")
        print(f"  Current Prices: Yes: {market['yes_ask']}/{market['yes_bid']}, No: {market['no_ask']}/{market['no_bid']}")
        print(f"  Volume: {market['volume']}")
        print(f"  Rules: {market['rules_primary']}")
        print()

Event details for KXHIGHLAX-25APR12:
Title: None
Category: None

All markets for LA temperature on April 12, 2025:
- Will the **high temp in LA** be >72° on Apr 12, 2025?
  Ticker: KXHIGHLAX-25APR12-T72
  Status: active
  Close Time: 2025-04-13T06:59:00Z
  Current Prices: Yes: 6/0, No: 100/94
  Volume: 0
  Rules: If the highest temperature recorded in Los Angeles Airport, CA for April 12, 2025 as reported by the National Weather Service's Climatological Report (Daily), is greater than 72°, then the market resolves to Yes.

- Will the **high temp in LA** be 71-72° on Apr 12, 2025?
  Ticker: KXHIGHLAX-25APR12-B71.5
  Status: active
  Close Time: 2025-04-13T06:59:00Z
  Current Prices: Yes: 16/10, No: 90/84
  Volume: 90
  Rules: If the highest temperature recorded in Los Angeles Airport, CA for April 12, 2025 as reported by the National Weather Service's Climatological Report (Daily), is between 71-72°, then the market resolves to Yes.

- Will the **high temp in LA** be 69-70° on Apr 12, 2

In [34]:
import requests

# Get details for the April 11 event
event_ticker = "KXHIGHLAX-25APR11"
url = f"https://api.elections.kalshi.com/trade-api/v2/events/{event_ticker}"
headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)
event_data = response.json()

print(f"Event details for {event_ticker}:")
print(f"Title: {event_data.get('title')}")
print(f"Category: {event_data.get('category')}")
print()

# Get details for each market in this event
markets_url = "https://api.elections.kalshi.com/trade-api/v2/markets"
markets_params = {"event_ticker": event_ticker}
markets_response = requests.get(markets_url, headers=headers, params=markets_params)
markets_data = markets_response.json()

if 'markets' in markets_data:
    print(f"All markets for LA temperature on April 11, 2025:")
    for market in markets_data['markets']:
        print(f"- {market['title']}")
        print(f"  Ticker: {market['ticker']}")
        print(f"  Status: {market['status']}")
        print(f"  Close Time: {market['close_time']}")
        print(f"  Current Prices: Yes: {market['yes_ask']}/{market['yes_bid']}, No: {market['no_ask']}/{market['no_bid']}")
        print(f"  Volume: {market['volume']}")
        print(f"  Rules: {market['rules_primary']}")
        print()
else:
    print("No markets found for the April 11 event or event doesn't exist")
    
    # If the direct approach fails, try searching for markets with APR11 in the ticker
    print("\nTrying alternative search for April 11 markets...")
    search_url = "https://api.elections.kalshi.com/trade-api/v2/markets"
    search_params = {"limit": 1000}
    search_response = requests.get(search_url, headers=headers, params=search_params)
    search_data = search_response.json()
    
    if 'markets' in search_data:
        apr11_markets = [
            market for market in search_data['markets'] 
            if "KXHIGHLAX-25APR11" in market['ticker'] or 
               "KXHIGHLAX-25APR11" in market['event_ticker'] or
               ("los angeles" in market['title'].lower() and "apr 11" in market['title'].lower())
        ]
        
        if apr11_markets:
            print(f"Found {len(apr11_markets)} LA temperature markets for April 11:")
            for market in apr11_markets:
                print(f"- {market['title']} (Ticker: {market['ticker']})")
                print(f"  Event Ticker: {market['event_ticker']}")
                print(f"  Status: {market['status']}")
                print(f"  Close Time: {market['close_time']}")
                print()
        else:
            print("No LA temperature markets found for April 11")

Event details for KXHIGHLAX-25APR11:
Title: None
Category: None

All markets for LA temperature on April 11, 2025:
- Will the **high temp in LA** be >76° on Apr 11, 2025?
  Ticker: KXHIGHLAX-25APR11-T76
  Status: active
  Close Time: 2025-04-12T06:59:00Z
  Current Prices: Yes: 3/0, No: 100/97
  Volume: 20331
  Rules: If the highest temperature recorded in Los Angeles Airport, CA for April 11, 2025 as reported by the National Weather Service's Climatological Report (Daily), is greater than 76°, then the market resolves to Yes.

- Will the **high temp in LA** be 75-76° on Apr 11, 2025?
  Ticker: KXHIGHLAX-25APR11-B75.5
  Status: active
  Close Time: 2025-04-12T06:59:00Z
  Current Prices: Yes: 99/94, No: 6/1
  Volume: 17396
  Rules: If the highest temperature recorded in Los Angeles Airport, CA for April 11, 2025 as reported by the National Weather Service's Climatological Report (Daily), is between 75-76°, then the market resolves to Yes.

- Will the **high temp in LA** be 73-74° on Apr 

In [35]:
import requests

# Let's try to find the Denver temperature markets for April 11
url = "https://api.elections.kalshi.com/trade-api/v2/markets"
headers = {"accept": "application/json"}
params = {"limit": 1000}  # Request maximum number of markets

response = requests.get(url, headers=headers, params=params)
data = response.json()

if 'markets' in data:
    # Look for markets related to Denver temperature on April 11
    denver_markets = [
        market for market in data['markets'] 
        if "denver" in market['title'].lower() and 
           "apr 11" in market['title'].lower()
    ]
    
    # Also look for markets with KXHIGHDEN-25APR11 in ticker or event ticker
    kxhighden_markets = [
        market for market in data['markets'] 
        if "KXHIGHDEN-25APR11" in market['ticker'] or 
           "KXHIGHDEN-25APR11" in market['event_ticker']
    ]
    
    # Combine and remove duplicates
    all_denver_markets = denver_markets + [m for m in kxhighden_markets if m not in denver_markets]
    
    if all_denver_markets:
        print(f"Found {len(all_denver_markets)} Denver temperature markets for April 11:")
        for market in all_denver_markets:
            print(f"- {market['title']}")
            print(f"  Ticker: {market['ticker']}")
            print(f"  Event Ticker: {market['event_ticker']}")
            print(f"  Status: {market['status']}")
            print(f"  Close Time: {market['close_time']}")
            print(f"  Current Prices: Yes: {market['yes_ask']}/{market['yes_bid']}, No: {market['no_ask']}/{market['no_bid']}")
            print(f"  Volume: {market['volume']}")
            print(f"  Rules: {market['rules_primary']}")
            print()
    else:
        print("No Denver temperature markets found for April 11")
        
        # Try looking for any Denver temperature markets (without date specificity)
        any_denver_markets = [
            market for market in data['markets'] 
            if "denver" in market['title'].lower() and 
               ("temperature" in market['title'].lower() or "temp" in market['title'].lower() or "high" in market['title'].lower())
        ]
        
        if any_denver_markets:
            print(f"Found {len(any_denver_markets)} Denver temperature markets (any date):")
            for market in any_denver_markets:
                print(f"- {market['title']} (Ticker: {market['ticker']})")
                print(f"  Event Ticker: {market['event_ticker']}")
else:
    print("No markets found or unexpected response format")

No Denver temperature markets found for April 11
Found 6 Denver temperature markets (any date):
- Will the **high temp in Denver** be >89° on Apr 12, 2025? (Ticker: KXHIGHDEN-25APR12-T89)
  Event Ticker: KXHIGHDEN-25APR12
- Will the **high temp in Denver** be 88-89° on Apr 12, 2025? (Ticker: KXHIGHDEN-25APR12-B88.5)
  Event Ticker: KXHIGHDEN-25APR12
- Will the **high temp in Denver** be 86-87° on Apr 12, 2025? (Ticker: KXHIGHDEN-25APR12-B86.5)
  Event Ticker: KXHIGHDEN-25APR12
- Will the **high temp in Denver** be 84-85° on Apr 12, 2025? (Ticker: KXHIGHDEN-25APR12-B84.5)
  Event Ticker: KXHIGHDEN-25APR12
- Will the **high temp in Denver** be 82-83° on Apr 12, 2025? (Ticker: KXHIGHDEN-25APR12-B82.5)
  Event Ticker: KXHIGHDEN-25APR12
- Will the **high temp in Denver** be <82° on Apr 12, 2025? (Ticker: KXHIGHDEN-25APR12-T82)
  Event Ticker: KXHIGHDEN-25APR12


In [36]:
import requests

# Get details for each market in this event
event_ticker = "KXHIGHDEN-25APR12"
markets_url = "https://api.elections.kalshi.com/trade-api/v2/markets"
headers = {"accept": "application/json"}
markets_params = {"event_ticker": event_ticker}
markets_response = requests.get(markets_url, headers=headers, params=markets_params)
markets_data = markets_response.json()

if 'markets' in markets_data:
    print(f"All markets for Denver temperature on April 12, 2025:")
    for market in markets_data['markets']:
        print(f"- {market['title']}")
        print(f"  Ticker: {market['ticker']}")
        print(f"  Status: {market['status']}")
        print(f"  Close Time: {market['close_time']}")
        print(f"  Current Prices: Yes: {market['yes_ask']}/{market['yes_bid']}, No: {market['no_ask']}/{market['no_bid']}")
        print(f"  Volume: {market['volume']}")
        print(f"  Open Interest: {market['open_interest']}")
        print(f"  Liquidity: {market['liquidity']}")
        print(f"  Rules: {market['rules_primary']}")
        print()

All markets for Denver temperature on April 12, 2025:
- Will the **high temp in Denver** be >89° on Apr 12, 2025?
  Ticker: KXHIGHDEN-25APR12-T89
  Status: active
  Close Time: 2025-04-13T05:59:00Z
  Current Prices: Yes: 7/6, No: 94/93
  Volume: 533
  Open Interest: 533
  Liquidity: 226216
  Rules: If the highest temperature recorded in Denver, CO for April 12, 2025 as reported by the National Weather Service's Climatological Report (Daily), is greater than 89°, then the market resolves to Yes.

- Will the **high temp in Denver** be 88-89° on Apr 12, 2025?
  Ticker: KXHIGHDEN-25APR12-B88.5
  Status: active
  Close Time: 2025-04-13T05:59:00Z
  Current Prices: Yes: 26/20, No: 80/74
  Volume: 33
  Open Interest: 33
  Liquidity: 242184
  Rules: If the highest temperature recorded in Denver, CO for April 12, 2025 as reported by the National Weather Service's Climatological Report (Daily), is between 88-89°, then the market resolves to Yes.

- Will the **high temp in Denver** be 86-87° on Ap

In [37]:
import requests

# Get details for each market in this event
event_ticker = "KXHIGHDEN-25APR13"
markets_url = "https://api.elections.kalshi.com/trade-api/v2/markets"
headers = {"accept": "application/json"}
markets_params = {"event_ticker": event_ticker}
markets_response = requests.get(markets_url, headers=headers, params=markets_params)
markets_data = markets_response.json()

if 'markets' in markets_data:
    print(f"All markets for Denver temperature on April 12, 2025:")
    for market in markets_data['markets']:
        print(f"- {market['title']}")
        print(f"  Ticker: {market['ticker']}")
        print(f"  Status: {market['status']}")
        print(f"  Close Time: {market['close_time']}")
        print(f"  Current Prices: Yes: {market['yes_ask']}/{market['yes_bid']}, No: {market['no_ask']}/{market['no_bid']}")
        print(f"  Volume: {market['volume']}")
        print(f"  Open Interest: {market['open_interest']}")
        print(f"  Liquidity: {market['liquidity']}")
        print(f"  Rules: {market['rules_primary']}")
        print()

All markets for Denver temperature on April 12, 2025:
- Will the **high temp in Denver** be >73° on Apr 13, 2025?
  Ticker: KXHIGHDEN-25APR13-T73
  Status: active
  Close Time: 2025-04-14T05:59:00Z
  Current Prices: Yes: 1/0, No: 100/99
  Volume: 3157
  Open Interest: 2467
  Liquidity: 131980
  Rules: If the highest temperature recorded in Denver, CO for April 13, 2025 as reported by the National Weather Service's Climatological Report (Daily), is greater than 73°, then the market resolves to Yes.

- Will the **high temp in Denver** be <66° on Apr 13, 2025?
  Ticker: KXHIGHDEN-25APR13-T66
  Status: active
  Close Time: 2025-04-14T05:59:00Z
  Current Prices: Yes: 1/0, No: 100/99
  Volume: 3477
  Open Interest: 2341
  Liquidity: 200606
  Rules: If the highest temperature recorded in Denver, CO for April 13, 2025 as reported by the National Weather Service's Climatological Report (Daily), is less than 66°, then the market resolves to Yes.

- Will the **high temp in Denver** be 72-73° on A